# Metaprogramming in TF 2.0:
- Meta programming is a programming technique where one program reads, compiles and analyzes another program during execution. Commonly used to shift computation from runtime to compile-time.
- TF 1.x relied heavily on metaprogramming
- metaprogramming is clunky and hard-to-use

In TF 2.0, for simple uses:
- Just go with dynamic computation graphs which are enabled by default.
- Write regualr python functions. (Works fine for almost all use-cases)

However, heavy duty use-cases still need metaprogramming:
- Static Computation Graphs are highly optimized.
- *tf.function* can be used

<ins>**tf.function**</ins>

Function decorator to convert python function (eager-execution) to graph generating code (static computation)

- Does the heavy-lifting of metaprogramming in TF 2.0
- Not needed at all except for specific use cases.
  - Distributed training and large models with large training datasets
- Reqrites Python control flow to TF control flow.
- Leverages GPUs and Cloud TPUs.
- Also known as *Autograph* and *just-in-time tracer*.
- Traces how python executes code.
  - Dynamic typing, polymorphism
- Separate graph for each type of input.
- Code with Python side-effects are executed only during the trace process.
- Tracing process produces graph representation.
- subsequent invocations to function executes graph
- Implemented in Autograph library


<ins>**Best Practices**</ins>

- Debug in eager mode, then decorate with tf.function.
- Don't rely on object mutation or list appends (Python side effects).
- tf.function works best with TF ops.
- Numpy and Python calls converted to constants.
- If Python function has side effects, do not decorate with tf.function.
- Beware of using tf.function with stateful functions
  - Generators, iterators
  

In [1]:
import time
import warnings
import logging
import tensorflow as tf

In [3]:
# @tf.function decorated functions run as static computation graphs

@tf.function
def add(x, y):
    return x + y

@tf.function
def sub(x, y):
    return x - y

@tf.function
def mul(x, y):
    return x * y

@tf.function
def div(x, y):
    return x / y

In [4]:
print(add(tf.constant(5), tf.constant(6)))
# Tracing computation graphs
# Only the first time a function is called, the computation graph is traced

tf.Tensor(11, shape=(), dtype=int32)


In [5]:
print(sub(tf.constant(5), tf.constant(6)))
print(mul(tf.constant(5), tf.constant(6)))
print(div(tf.constant(5), tf.constant(6)))

tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(30, shape=(), dtype=int32)
tf.Tensor(0.8333333333333334, shape=(), dtype=float64)


In [6]:
@tf.function
def matmul(x, y):
    return tf.matmul(x, y)

In [7]:
@tf.function
def linear(m, x, c):
    return add(matmul(m, x), c)

In [8]:
m = tf.constant([[4.0, 5.0, 6.0]], tf.float32)
m

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4., 5., 6.]], dtype=float32)>

In [9]:
x = tf.Variable([[100.0], [100.0], [100.0]], tf.float32)
x

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[100.],
       [100.],
       [100.]], dtype=float32)>

In [10]:
c = tf.constant([[1.0]], tf.float32)
c

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>

In [11]:
linear(m, x, c)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1501.]], dtype=float32)>

In [14]:
@tf.function
def pos_neg_check(x):
    reduce_sum = tf.reduce_sum(x)

    if reduce_sum > 0:
        return tf.constant(1)
    elif reduce_sum == 0:
        return tf.constant(0)
    else:
        return tf.constant(-1)

In [15]:
pos_neg_check(tf.constant([100, 100]))

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [16]:
pos_neg_check(tf.constant([-100, -100]))

<tf.Tensor: shape=(), dtype=int32, numpy=-1>

In [17]:
pos_neg_check(tf.constant([100, -100]))

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [18]:
num = tf.Variable(7)

In [19]:
@tf.function
def add_times(x):
    for i in tf.range(x):
        num.assign_add(x)

In [20]:
print(num)
add_times(5)
print(num)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=32>


In [21]:
a = tf.Variable(1.0)
b = tf.Variable(2.0)

In [22]:
@tf.function
def f(x, y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

In [23]:
print(f'a = {a.numpy()}, b = {b.numpy()}')
print(f(1, 2))
print(f'a = {a.numpy()}, b = {b.numpy()}')

a = 1.0, b = 2.0
tf.Tensor(10.0, shape=(), dtype=float32)
a = 4.0, b = 6.0


In [27]:
@tf.function
def square(x):
    print(f'Input = {x}') # Python side-effect => Only runs during tracing
    return x * x

In [28]:
x = tf.Variable([[2, 2], [2, 2]], tf.float32)
print(square(x)) # Print statement inside the function is executed only once during the tracing

Input = <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>
tf.Tensor(
[[4 4]
 [4 4]], shape=(2, 2), dtype=int32)


In [32]:
print(square(x)) # Print statement is not executed again for subsequent calls

tf.Tensor(
[[4 4]
 [4 4]], shape=(2, 2), dtype=int32)


In [31]:
y = tf.Variable([[2, 2], [2, 2]], tf.int32) # Separate graph is created for int32
print(square(y)) 

Input = <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>
tf.Tensor(
[[4 4]
 [4 4]], shape=(2, 2), dtype=int32)


In [33]:
z = tf.Variable([[2, 2], [2, 2]], tf.int32) # Separate graph is created for another variable
print(square(z))

Input = <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>
tf.Tensor(
[[4 4]
 [4 4]], shape=(2, 2), dtype=int32)


In [34]:
z1 = tf.Variable([[2, 2], [2, 2]], tf.float32)
square(z1)

Input = <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]])>

In [35]:
square(tf.constant([[2, 2], [2, 2]]))

Input = Tensor("x:0", shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]])>

In [36]:
square(tf.constant([[2, 2], [2, 2]])) # New graphs are not created for separate constants with same type/signatures

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]])>

In [37]:
square(tf.Variable([[2, 2], [2, 2]]))

Input = <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]])>

In [38]:
square(tf.Variable([[2, 2], [2, 2]]))

Input = <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]])>

In [39]:
concrete_int_square_fn = square.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.int32))
concrete_int_square_fn

Input = Tensor("x:0", dtype=int32)


<ConcreteFunction square(x) at 0x25AC72BA970>

In [40]:
concrete_float_square_fn = square.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.float32))
concrete_float_square_fn

Input = Tensor("x:0", dtype=float32)


<ConcreteFunction square(x) at 0x25AC703A0A0>

In [41]:
concrete_int_square_fn(tf.constant([[2, 2], [2, 2]], tf.int32))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]])>

In [42]:
concrete_float_square_fn(tf.constant([[2, 2], [2, 2]], tf.float32))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[4., 4.],
       [4., 4.]], dtype=float32)>

In [43]:
concrete_float_square_fn(tf.constant([[2, 2], [2, 2]], tf.int32))

InvalidArgumentError: cannot compute __inference_square_376 as input #0(zero-based) was expected to be a float tensor but is a int32 tensor [Op:__inference_square_376]

In [44]:
@tf.function
def f(x):
    print(f'Python execution: x = {x}')
    tf.print(f'TensorFlow execution: x = {x}')

In [45]:
f(1)

Python execution: x = 1
TensorFlow execution: x = 1


In [47]:
f(1)

TensorFlow execution: x = 1


In [48]:
f('Hello')

Python execution: x = Hello
TensorFlow execution: x = Hello


In [49]:
f(tf.constant(1))

Python execution: x = Tensor("x:0", shape=(), dtype=int32)
TensorFlow execution: x = Tensor("x:0", shape=(), dtype=int32)


In [50]:
f(tf.constant(2))

TensorFlow execution: x = Tensor("x:0", shape=(), dtype=int32)


In [51]:
def fn_with_variable_init_eager():

    a = tf.constant([[10, 10], [11., 1.]])
    x = tf.constant([[1., 0.], [0., 1.]])
    b = tf.Variable(12.)

    y = tf.matmul(a, x) + b
    tf.print(f'tf_print: {y}')

    return y

In [53]:
fn_with_variable_init_eager()

tf_print: [[22. 22.]
 [23. 13.]]


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

In [54]:
@tf.function
def fn_with_variable_init_autograph():

    a = tf.constant([[10, 10], [11., 1.]])
    x = tf.constant([[1., 0.], [0., 1.]])
    b = tf.Variable(12.)

    y = tf.matmul(a, x) + b
    tf.print(f'tf_print: {y}')

    return y

In [55]:
fn_with_variable_init_autograph()

ValueError: in user code:

    C:\Users\SAFIUD~1\AppData\Local\Temp/ipykernel_23804/2427516161.py:6 fn_with_variable_init_autograph  *
        b = tf.Variable(12.)
    C:\Users\Safiuddin\anaconda3\envs\tfenv\lib\site-packages\tensorflow\python\ops\variables.py:268 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    C:\Users\Safiuddin\anaconda3\envs\tfenv\lib\site-packages\tensorflow\python\ops\variables.py:250 _variable_v2_call
        return previous_getter(
    C:\Users\Safiuddin\anaconda3\envs\tfenv\lib\site-packages\tensorflow\python\ops\variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    C:\Users\Safiuddin\anaconda3\envs\tfenv\lib\site-packages\tensorflow\python\eager\def_function.py:764 invalid_creator_scope
        raise ValueError(

    ValueError: tf.function-decorated function tried to create variables on non-first call.


In [56]:
class F():

    def __init__(self):
        self._b = None

    
    @tf.function
    def __call__(self):
        a = tf.constant([[10, 10], [11., 1.]])
        x = tf.constant([[1., 0.], [0., 1.]])

        if self._b is None:
            self._b = tf.Variable(12.)

        y = tf.matmul(a, x) + self._b
        tf.print(f'tf_print: {y}')

        return y

In [57]:
fn_with_variable_init_autograph = F()
fn_with_variable_init_autograph()

tf_print: Tensor("add:0", shape=(2, 2), dtype=float32)


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

In [58]:
def f(x):
    if x > 0:
        x *= x
    return x

print(tf.autograph.to_code(f))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def if_body():
            nonlocal x
            x = ag__.ld(x)
            x *= x

        def else_body():
            nonlocal x
            pass
        ag__.if_stmt(ag__.ld(x) > 0, if_body, else_body, get_state, set_state, ('x',), 1)
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [67]:
@tf.function
def g(x):
    return x

start = time.time()
for i in tf.range(2000):
    g(i)
end = time.time()

print(f'tf.Tensor time elapsed with tf.range: {end - start}')

tf.Tensor time elapsed with tf.range: 0.9736378192901611


In [68]:
warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').disabled = True
start = time.time()
for i in range(2000):
    g(i)
end = time.time()

print(f'Native type time elapsed: {end - start}')

Native type time elapsed: 21.81544589996338
